In [1]:
import numpy as np
import pandas as pd
import torch
import torchaudio
from torch.utils.data import DataLoader
import classes.ProjectClasses as pcl

In [2]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NVIDIA GeForce RTX 3060


In [3]:
ANNOTATIONS_FILE_TEST = "../test_data/metadata/file_labels.csv"
AUDIO_DIR_TEST = "../test_data/files"
SAMPLE_RATE = 10000
NUM_SAMPLES = 9600
N_FFT = 400
HOP_LENGTH = 160
N_MELS = 64
LABEL_COL = 1
NUM_CLASSES = 20

tt_utils = pcl.TrainTestUtils(NUM_CLASSES)
mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate = SAMPLE_RATE, n_fft = N_FFT, hop_length = HOP_LENGTH, n_mels = N_MELS)
spr_test = pcl.SpikerboxRecordings(ANNOTATIONS_FILE_TEST, AUDIO_DIR_TEST, mel_spectrogram, SAMPLE_RATE, NUM_SAMPLES, device, LABEL_COL)

In [4]:
VGGish_Grade = pcl.VGGishNetwork(NUM_CLASSES).to(device)
VGGish_Grade.load_state_dict(torch.load('../trained_models/VGGish_Grade.pth', weights_only = False))

<All keys matched successfully>

In [5]:
BATCH_SIZE = 128
generator = torch.Generator()
generator.manual_seed(569567390)
loss_fn = torch.nn.CrossEntropyLoss().to(device)
test_dl = DataLoader(spr_test, batch_size = BATCH_SIZE, shuffle = True, generator = generator)
tt_utils.test_model(VGGish_Grade, test_dl, loss_fn, device)

[Test Results]
Test Loss: 7.8880745, Test Accuracy: 58.31%, Scaled Test Accuracy: 93.86%


In [6]:
df_metadata_test = pd.DataFrame(pd.read_csv(ANNOTATIONS_FILE_TEST))
exam_names = np.unique([element.split('_')[0] for element in df_metadata_test.filename.values])
exams = list()
for exam_name in exam_names:
    exam_labels = df_metadata_test[df_metadata_test.filename.str.startswith(exam_name)]
    spr_test_single_exam = pcl.SpikerboxRecordings(exam_labels, AUDIO_DIR_TEST, mel_spectrogram, SAMPLE_RATE, NUM_SAMPLES, device, LABEL_COL)
    exams.append(spr_test_single_exam)
for exam in exams:
    test_dl = DataLoader(exam, batch_size = len(exam), shuffle = True)
    prediction = tt_utils.evaluate_exam(VGGish_Grade, test_dl, device)
    print("Predicted Grade Bin:", prediction)
    print("Actual Grade Bin:", exam[0][1])

Predicted Grade Bin: 8
Actual Grade Bin: 8
Predicted Grade Bin: 6
Actual Grade Bin: 7
Predicted Grade Bin: 7
Actual Grade Bin: 4
